<a href="https://colab.research.google.com/github/W-Mrt/Machine-Learning-for-Finance/blob/main/FIN_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow-gpu
#!pip install tensorflow
# !pip install keras
# !pip install matplotlib
# !pip install pandas
# !pip install numpy

In [23]:
#main
import tensorflow as tf
#from tensorflow import set_random_seed
from tensorflow.keras.utils import set_random_seed
#from fin_data import data_generator
from keras.models import Sequential
# from keras.layers.merge import Add
from keras.layers import Add
from keras.optimizers import Adam,Nadam
#from models import discriminator_model, generator_model_cnn,generator_model_mlp,generator_model_mlp_cnn,generator_model_mlp_cnn_plus
import matplotlib.pyplot as plt
#from stats import acf
import numpy as np
from numpy.random import seed
#import visualize
#import stylized_facts as sf
from datetime import datetime as dt
import os
from shutil import copyfile
import argparse 

In [25]:
#stats
import numpy as np
import pandas as pd

def auto_correlation(x,lag = 1):
	a = pd.Series(np.reshape(x,(-1)))
	b = a.autocorr(lag = lag)
	if np.isnan(b) or np.isinf(b):
		return 0
	return b

def acf(x,max_lag=1000):
	acf = []
	for i in range(max_lag):
		acf.append(auto_correlation(x,lag=i+1))
	return np.array(acf)

In [27]:
#visualizer
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
#import stats
import numpy as np

def matplotlib_config():
		mpl.rc('text', color='#3ba6d8')
		mpl.rc('axes', facecolor='none',edgecolor="#3ba6d8",labelcolor="#3ba6d8",titlesize=15,labelsize=10,grid=True)
		mpl.rc('lines', linewidth=1,color="#3ba6d8")
		mpl.rc('grid',color='#f2f2f2',alpha=0.3,linewidth=1)
		mpl.rc('xtick',color='#3ba6d8')
		mpl.rc('ytick',color='#3ba6d8')

def basic_graph(x,title=''):
	matplotlib_config()
	plt.figure()
	plt.plot(x,color='red')
	plt.legend(handles=stat_patch_list(x),fontsize=10,loc='upper right')
	#plt.ylim(-0.5,1.0)
	plt.savefig(title +'.png',transparent=True)
	plt.close()

def time_series_graph(x,title=''):
	matplotlib_config()
	plt.figure()
	plt.plot(x,color='red')
	plt.savefig(title +'.png',transparent=True)
	plt.close()

def graph_with_subsampling(x,subsample_list,title=''):
	matplotlib_config()
	plt.figure()
	plt.plot(x,color='red')
	for sample in subsample_list:
		plt.plot(sample,color='blue')
	plt.legend(handles=stat_patch_list(x),fontsize=10,loc='upper right')
	plt.savefig(title +'.png',transparent=True)
	plt.close()

def prediction_graph(pred,y,title=''):
	matplotlib_config()
	plt.figure()
	plt.plot(y,color='blue')
	plt.plot(pred,color='red')
	plt.title(title)
	#plt.xlabel('data in series')
	#plt.ylabel('std21')
	plt.legend(handles=comparing_stat_patch_list(pred,y),fontsize=10,loc='upper right')
	plt.savefig(title +'.png',transparent=True)
	plt.close()

def histogram_graph(y,title=''):
	matplotlib_config()
	plt.figure()
	plt.hist(y,bins=100,color='blue')
	plt.title(title)
	plt.legend(handles=stat_patch_list(y),fontsize=10,loc='upper right')
	plt.savefig(title +'.png',transparent=True)
	plt.close()

def comparing_stat_patch_list(pred,y):
	patch1 = mpatches.Patch(color='red', label= 'mean:'+ ('%03.6f' % np.mean(pred)))
	patch2 = mpatches.Patch(color='red', label= 'std:'+ ('%03.6f' % np.std(pred)))
	patch3 = mpatches.Patch(color='red', label= 'skewness:'+ ('%03.3f' % stats.skewness(pred)))
	patch4 = mpatches.Patch(color='red', label= 'kurtosis:'+ ('%03.3f' % stats.kurtosis(pred)))
	patch5 = mpatches.Patch(color='blue', label= 'mean:'+ ('%03.6f' % np.mean(y)))
	patch6 = mpatches.Patch(color='blue', label= 'std:'+ ('%03.6f' % np.std(y)))
	patch7 = mpatches.Patch(color='blue', label= 'skewness:'+ ('%03.3f' % stats.skewness(y)))
	patch8 = mpatches.Patch(color='blue', label= 'kurtosis:'+ ('%03.3f' % stats.kurtosis(y)))
	#patch9 = mpatches.Patch(color='black', label= 'MAPE:'+ ('%03.3f' % stats.mape(pred, y)))
	patch10 = mpatches.Patch(color='black', label= 'RMSE:'+ ('%03.6f' % stats.rmse(pred, y)))
	#plt.text(.25,.5,str(np.mean(pred)))
	return [patch5,patch6,patch7,patch8,patch1,patch2,patch3,patch4,patch10]

def stat_patch_list(x):
	patch0 = mpatches.Patch(label= 'data num:'+ str((x.size)))
	patch1 = mpatches.Patch(label= 'mean:'+ ('%03.6f' % np.mean(x)))
	patch2 = mpatches.Patch(label= 'std:'+ ('%03.6f' % np.std(x)))
	patch3 = mpatches.Patch(label= 'skewness:'+ ('%03.3f' % stats.skewness(x)))
	patch4 = mpatches.Patch(label= 'kurtosis:'+ ('%03.3f' % stats.kurtosis(x)))
	return [patch0,patch1,patch2,patch3,patch4]

def basic_patch_list():
	patch1 = mpatches.Patch(color='red', label= 'prediciton')
	patch2 = mpatches.Patch(color='blue', label= 'actual value')
	return [patch1,patch2]

def patch_list(pred,y):
	return stat_patch_list(pred, y)

In [28]:
#data
import pandas as pd
import numpy as np
#import arch
import os
#import stats
#import visualizer
from datetime import datetime
import warnings
#import talib as ta

warnings.filterwarnings('ignore')

class data_manager():
	def __init__(self):
		self.csv_file = ''
		pass

#where to normalize?
	def prepare_pd(self):
		self.read_csv()
		self.parse_date()
		self.data_begin_date = None
		self.data_end_date = None
		self.add_columns()

	def add_columns(self,is_financial = True, contain_adjusted = True,contain_volume=True,validation_split=0.81):
		if is_financial:
			if contain_adjusted:
				self.data['Log Return'] = np.log(self.data['Adj Close']) - np.log(self.data['Adj Close'].shift(1))
				self.data['Pct Change'] = self.data['Adj Close'].pct_change().dropna()
			else:
				self.data['Log Return'] = np.log(self.data['Close']) - np.log(self.data['Close'].shift(1))
				self.data['Pct Change'] = self.data['Close'].pct_change().dropna()
			# self.data['U'] = np.log(self.data['High']/self.data['Open'])
			# self.data['D'] = np.log(self.data['Low']/self.data['Open'])
			# self.data['C'] = np.log(self.data['Close']/self.data['Open'])
			# self.data['A'] = self.data.apply(lambda x: 1 if x['Log Return'] >=0 else 0 ,axis=1)
			# self.data['Y'] = self.data['A'].shift(-1)
			# self.data['Daily Volatility'] = np.sqrt(0.511*((self.data['U']-self.data['D'])**2) - 0.019*(self.data['C']*(self.data['U']+self.data['D'])-2*self.data['U']*self.data['D']) - 0.383*(self.data['C']**2))
			# self.data['Simple 10-day MA'] = pd.rolling_mean(self.data['Close'],10)
			# self.data['Weighted 10-day MA'] = ta.WMA(np.array(self.data['Close'],dtype=np.float),timeperiod=10)
			# self.data['Momentum'] = ta.MOM(np.array(self.data['Close'],dtype=np.float), timeperiod=10)
			# self.data['Stochastic K'] = stats.stochastic_K(self.data['Close'], self.data['High'], self.data['Low'], 14)
			# self.data['Stochastic D'] = stats.stochastic_D(self.data['Close'], self.data['High'], self.data['Low'], 14,3)
			# self.data['sign'] = self.data.apply(lambda x: 1 if x['Log Return'] >=0 else -1 ,axis=1)
			# self.data['RSI'] = ta.RSI(np.array(self.data['Close'],dtype=np.float), timeperiod=14)
			# self.data['MACD'] = ta.MACD(np.array(self.data['Close'],dtype=np.float),fastperiod=12, slowperiod=26, signalperiod=9)[0]
			# self.data['Larry William R'] = ta.WILLR(np.array(self.data['High'],dtype=np.float), np.array(self.data['Low'],dtype=np.float),np.array(self.data['Close'],dtype=np.float), timeperiod=14)
			# self.data['AD'] = stats.AD(self.data['Close'],self.data['High'],self.data['Low'])
			# self.data['CCI'] = ta.CCI(np.array(self.data['Close'],dtype=np.float), np.array(self.data['Low'],dtype=np.float), np.array(self.data['High'],dtype=np.float), timeperiod=20)
			# if contain_volume:
			# 	self.data['Mean Volume 300'] = pd.rolling_mean(self.data['Volume'], 300)
			# 	self.data['Std Volume 300'] = pd.rolling_std(self.data['Volume'], 300)
			# 	self.data['Normalized Volume 300'] = (self.data['Volume']-self.data['Mean Volume 300'])/self.data['Std Volume 300']
				#self.data['Mean Volume 1000'] = pd.rolling_mean(self.data['Volume'], 1000)
				#self.data['Std Volume 1000'] = pd.rolling_std(self.data['Volume'], 1000)
				#self.data['Normalized Volume 1000'] = (self.data['Volume']-self.data['Mean Volume 1000'])/self.data['Std Volume 1000']
			# self.data['Stdev2'] = pd.rolling_std(self.data['Log Return'], 2)
			# self.data['Stdev7'] = pd.rolling_std(self.data['Log Return'], 7)
			# self.data['Stdev14'] = pd.rolling_std(self.data['Log Return'], 14)
			# self.data['Stdev21'] = pd.rolling_std(self.data['Log Return'], 21)
			# self.data['Stdev28'] = pd.rolling_std(self.data['Log Return'], 28)
			#GARCH fitting
			self.data = self.data.dropna()
			#
			# self.data['Forecast Volatility'] = np.sqrt(res.params['omega'] + res.params['alpha[1]'] * res.resid**2 + res.conditional_volatility**2 * res.params['beta[1]'])
			# self.data['Annualized Forecast Volatility'] = self.data['Forecast Volatility']*np.sqrt(252)

	def fill_garch(self,ret,validation_split=0.2):
		c = 100
		am = arch.arch_model(ret[:int((1-validation_split)*ret.size)]*c)
		res = am.fit(update_freq=5,disp='on')
		new_cond_vol = np.zeros(ret.size)
		c_vol = res.conditional_volatility
		for i in range(ret.size):
			if c_vol.size > i:
				new_cond_vol[i] = c_vol[i]
			else:
				new_cond_vol[i] = np.sqrt(res.params['omega'] + res.params['alpha[1]'] * ((ret[i-1]*c)-res.params['mu'])**2 + (new_cond_vol[i - 1]**2)*res.params['beta[1]'])
		return new_cond_vol/c
			#return None
	def read_csv(self):
		self.data = pd.read_csv(self.csv_file)

	def data_stat(self):
		pass

	def get_pd_table(self,**kwargs):
		table = self.data
		if 'date_start' in kwargs:
			table = table.ix[table['Date'] > kwargs['date_start']]
			kwargs.pop('date_start')
		if 'date_end' in kwargs:
			table = table.ix[self.data['Date'] < kwargs['date_end']]
			kwargs.pop('date_end')

		return table

	def get_dataset(self,column_list,look_back = 10,normalize_width = 0,normalize_scheme = None,train_ratio = 0.8):
		#get specified pd table
		specified_table = self.data(column_list)
		#Normalize
		if normalize_width:
			for e in normalize_scheme:
				pass
		#Look-Back

		#Divide data into train/test
		train_x = None
		train_y = None
		test_x = None
		test_y = None
		return self._create_dataset(train_x,train_y),self._create_dataset(test_x,test_y)
		pass

	def create_dataset(self,x_set, y_set,look_back=10):
		data_x, data_y = [], []
		for i in range(len(x_set)-look_back-1):
			a = x_set[i:(i+look_back)]
			data_x.append(a)
			data_y.append(y_set[i + look_back - 1])
		return np.array(data_x), np.array(data_y)

	def parse_date(self):
		self.data['Date'] = pd.to_datetime(self.data['Date'])
		oldtime_table = self.data.copy()
		oldtime_table['Date'] -= pd.Timedelta(100,'Y')
		self.data = self.data.where(self.data['Date'] < pd.to_datetime('2017/1/1'),oldtime_table)

class gdc(data_manager):
	def __init__(self):
		super().__init__()
		self.csv_file = ''

	def prepare_pd(self):
		self.data = pd.read_csv(self.csv_file)


class snp500(data_manager):
	def __init__(self):
		self.csv_file = './data/SNP500/snp500.csv'

	def get_code_list(self):
		t = pd.read_csv('./data/SNP500/SNP500_Individuals/constituents.csv')
		return t['Symbol'].as_matrix()


class snp500_individual(data_manager):
	def __init__(self,code):
		#os.chdir("./data/SNP500/SNP500_Individuals")
		self.csv_file = "./data/SNP500/SNP500_Individuals/" +  str(code) + '.csv'

	def read_csv(self):
		self.data = pd.read_csv(self.csv_file,dtype={'Open': np.float32,'Low': np.float32,'High': np.float32,'Close': np.float32,'Volume': np.float32,'Adj_Close': np.float32})
		self.data.rename(columns={'Adj_Close':'Adj Close'},inplace=True)
		#self.data = self.data.iloc[::-1]



class nikkei225_individual(data_manager):
	def __init__(self,code):
		#os.chdir("./data/Nikkei225/Nikkei225_Individuals")
		self.csv_file = "../data/Nikkei225/Nikkei225_Individuals/" + str(code) + '.csv'

	def read_csv(self):
		self.data = pd.read_csv(self.csv_file,names=['Date','Open','High','Low','Close','Volume','Adj Close'],dtype={'Open': np.float32,'Low': np.float32,'High': np.float32,'Close': np.float32,'Volume': np.float32,'Adj Close': np.float32})
		self.data = self.data.iloc[::-1]

class nikkei225(data_manager):
	def __init__(self):
		#os.chdir("./data/Nikkei225")
		self.csv_file = '../data/Nikkei225/nikkei 225.csv'

	def prepare_pd(self):
		self.read_csv()
		self.data_begin_date = None
		self.data_end_date = None
		self.add_columns(True,False,False)

	def read_csv(self):
		#self.data = pd.read_csv(self.csv_file,names=['Date','Open','Low','High','Close'],dtype={'Open': np.float32,'Low': np.float32,'High': np.float32,'Close': np.float32})
		self.data = pd.read_csv(self.csv_file)
		self.data = self.data.rename(index=str, columns={"date": "Date", "open": "Open","low": "Low","high": "High","close": "Close"})

	def get_code_list(self):
		t = pd.read_csv("../data/Nikkei225/Nikkei225_Individuals/nikkei225-stock-prices.csv")
		return t['SC'].as_matrix()

class djia(data_manager):
	def __init__(self):
		super().__init__()
		self.csv_file = ''

In [29]:
#fin_data
import sys
#sys.path.append('./lib')
import numpy as np
#from data import snp500,snp500_individual

# s.prepare_pd()
# table = s.get_pd_table()
# size = table['Log Return'].as_matrix().size
# k = table['Log Return'].as_matrix()

class data_generator():
        def __init__(self):
                self.sequence_length = 1024*8
                self.batch_size = 12
                self.learning_phase = "train"
                self.train_split = 0.8

        def snp500_index(self):
                s = snp500()
                s.prepare_pd()
                table = s.get_pd_table()
                return table['Log Return'].as_matrix()
        def real_data(self,mode = "individuals"):
                if mode == "individuals":
                        return self.individuals_data_random_picker()
                elif mode == "index":
                        return self.index_data_random_picker()

        def individuals_data_random_picker(self):
                data = []
                for i in range(self.batch_size):
                        random_code = self.choose_random_code()
                        si = snp500_individual(random_code)
                        si.prepare_pd()
                        table = si.get_pd_table()
                        sequence = table['Log Return'].as_matrix()
                        size = table['Log Return'].as_matrix().size
                        random_pos = np.random.randint(0,size-self.sequence_length)
                        data.append([sequence[random_pos:random_pos+self.sequence_length]])
                data = np.array(data)
                #data /= max(data.max(),-data.min())
                data = np.reshape(data,(self.batch_size,self.sequence_length,1))
                return data

        def choose_random_code(self):
                s = snp500()
                codes = s.get_code_list()
                codes_size = len(codes)
                size = 0
                while size < self.sequence_length:
                        if self.train_split == -1:
                                random_code = codes[np.random.randint(0,codes_size)]
                        else:
                                if self.learning_phase == "train":
                                        random_code = codes[np.random.randint(0,int(self.train_split*codes_size))]
                                else:
                                        random_code = codes[np.random.randint(int(self.train_split*codes_size),codes_size)]
                        si = snp500_individual(random_code)
                        si.prepare_pd()
                        table = si.get_pd_table()
                        size = table['Log Return'].as_matrix().size
                return random_code

        def index_data_random_picker(self):
                s = snp500()
                s.prepare_pd()
                table = s.get_pd_table()
                sequence = table['Log Return'].as_matrix()
                size = table['Log Return'].as_matrix().size
                data = []
                for i in range(self.batch_size):
                        random_pos = np.random.randint(0,size-self.sequence_length)
                        data.append([sequence[random_pos:random_pos+self.sequence_length]])
                data = np.array(data)
                #data /= max(data.max(),-data.min())
                data = np.reshape(data,(self.batch_size,self.sequence_length,1))
                return 

In [30]:
parser = argparse.ArgumentParser(description='FIN-GAN implementation')
parser.add_argument('--batch_size',type=int,default=24)
parser.add_argument('--generator_model',type=str,default='mlp-cnn')
parser.add_argument('--epochs',type=int,default=10)
parser.add_argument('--batches',type=int,default=1024)
parser.add_argument('--folder_name',type=str,default='')
parser.add_argument('--generator_lr',type=float,default='2e-4')
parser.add_argument('--discriminator_lr',type=float,default='1e-5')
parser.add_argument('--log_interval',type=int,default=50)
parser.add_argument('--seed',type=int,default=1)
parser.add_argument('-f')

args = parser.parse_args()
seed(args.seed)
set_random_seed(args.seed)

In [44]:
dg = data_generator()
batch_size = 24
dg.batch_size = batch_size
batches = 1024
epochs = 10                 
timestamp = dt.now().strftime("%Y%m%d_%H%M%S")
timestamp += '_'
timestamp += args.folder_name
#os.makedirs('./imgs/%s'%(timestamp), exist_ok=True)
os.mkdir('./imgs/%s'%(timestamp))
#os.makedirs('./npy/%s'%(timestamp), exist_ok=True)
os.mkdir('./npy/%s'%(timestamp))
#os.makedirs('./weights/%s'%(timestamp), exist_ok=True)
os.mkdir('./weights/%s'%(timestamp))
#os.makedirs('./imgs/%s/acf'%(timestamp), exist_ok=True)
os.mkdir('./imgs/%s/acf'%(timestamp))
#os.makedirs('./imgs/%s/dist'%(timestamp), exist_ok=True)
os.mkdir('./imgs/%s/dist'%(timestamp))
#os.makedirs('./imgs/%s/time_series'%(timestamp), exist_ok=True)
os.mkdir('./imgs/%s/time_series'%(timestamp))
#os.makedirs('./imgs/%s/leverage'%(timestamp), exist_ok=True)
os.mkdir('./imgs/%s/leverage'%(timestamp))
copyfile('./FIN-GAN.ipynb','./imgs/%s/main.py'%(timestamp))
#copyfile('./models.py','./imgs/%s/models.py'%(timestamp))
with open('./imgs/%s/hyper_parameters.txt'%(timestamp),'w') as w:
    w.write(str(args))

FileNotFoundError: ignored